# 01 element perturbation

Within the following tutorial you are going to see how to perturb one element of our TOY graph.

### Input description

The input for the graph construction currently consists of text files reflecting the hierarchy of the plant elements and their features. In the text input files each line corresponds to a node/element description.
The same line reports the name of the predecessor of a particular node/element, the relationship between them, and the list of node’s attributes (area in which the element is 
present, perturbation resistance, etc.).
In this way each line corresponds to an edge connecting a element to its parent element.

Each line should contain the following info:
- element id (**mark**)
- parent-child relationship (**father_condition**: *AND*, *OR*, *SINGLE*, *ORPHAN*. It is an edge attribute.)
- id of the parent of the element (**father_mark**)
- area in which the element is located (**area**. It is a node attribute.)
- passive perturbation resistance (**perturbation_resistant**: *1*, *0*. It is a node attribute.)
- state of the SWITCH(es) (**init_status**: *1*, *0*. It is a node attribute.)
- description of the element (**description**. It is a node attribute.)
- source - hub - switch - user elements (**type**: *SOURCE* or *HUB* or *SWITCH* or *USER*. It is a node attribute.)
- weight of the edge between element and parent (**weight**: float number. It is an edge attribute.)
- initial service at the sources (**initial_service**. It is a node attribute, for the moment valid just for sources, and 0. for other nodes. It is a node attribute.)

The hierarchy of the elements explains how commodities flow from one element to another element and from one system to another system. In fact, if the input is properly formatted, with this simple digraph model it is possible to represent and integrate different interconnected plants in a unique graph without losing information about their peculiarities. 

In the graph, the nodes represent the system elements while the edges connecting the nodes harbor the logic relations (edge attributes) existing between the elements (*ORPHAN*, *SINGLE*, *AND*, and *OR*):
- An **ORPHAN** edge is the edge of a node without predecessors.
- A **SINGLE** edge connects a node to its only one predecessor.
- An **AND** edge indicates that the node/element has more than one predecessor. All the predecessors are necessary for the functioning of that element.
- An **OR** edge indicates that the node/element has more than one predecessor. Just one of the node’s predecessors should be active to guarantee the functioning of the element. For this reason, correct input formatting is one of the most important steps of the analysis.

### Example

In the cartoon below it is represented the example input file `TOY_graph.csv`.
In this file are present 19 nodes/elements connected by direct edges that reflect the hierarchy of the system in a parent-child fashion.

The nodes are distributed in adjacent areas.

In area1 are present 5 nodes: 1, 2, 3, 4 and 5.

In area2 are present nodes: 11, 19, 12, 13, 14 and 18.

In area3 are present nodes: 15, 9, 16, 17 and 10.

In area4 are present nodes: 6, 7 and 8.

A perturbation of one or multiple elements in one area may exceed the area boundaries and propagate to other systems connected to it, located in other areas. 

Nodes 2, 3, 4, 5 are perturbation resistant nodes (`perturbation_resistant` field = 1).
These nodes will not be affected by the simulated perturbation.

Nodes 2 and 3 are isolating elements (they are a particular type of `HUB` nodes, called `SWITCH`). In the figure, perturbing node 1 would result in the breakage of all the nodes present in the graph except 
node 15 in absence of perturbation resistant nodes. On the other hand, isolating elements 2 and 3 would stop the perturbation propagation cascade to node 1.

<img src="./input_files/TOY_graph.png" alt="TOY_graph" width="800" height="600"/>

### Simulate a perturbation to element "1"

Let us see how to generate a perturbation that propagates from element "1" of the TOY graph. First of all, we need to import GRAPE classes. We are also importing `pandas` package since we are going to use it for reading the `CSV` output files obtained.

In [1]:
import pandas as pd
from grape.general_graph import GeneralGraph
from grape.fault_diagnosis import FaultDiagnosis

Second of all, we define a FaultDiagnosis class and we load the nodes for the TOY graph from the input file.

In [2]:
F = FaultDiagnosis("./input_files/TOY_graph.csv")

Let us check the input before perturbing the graph in any way. In order to do so we use `check_input_with_gephi` function on F, the `FaultDiagnosis` class, which prints out the list of edges and the list of nodes of the graph.  The two output files generated can be used to visualize the input with Gephi.

In [3]:
F.check_input_with_gephi()

Let us have a look at the list of nodes and edges that get printed for the integer TOY graph.

In [4]:
nodes = pd.read_csv("check_import_nodes.csv", na_filter=False)
nodes

mark description init_status  perturbation_resistant   area
0      1                                               0  area1
1      2                       1                       1  area1
2      3                       1                       1  area1
3      4                                               1  area1
4      5                                               1  area1
5      6                                               0  area4
6      7                                               0  area4
7      8                                               0  area4
8      9                                               0  area3
9     15                                               0  area3
10    16                                               0  area3
11    17                                               0  area3
12    10                                               0  area3
13    11                                               0  area2
14    19                                               0  area2
15    12                                               0  area2
16    13                                               0  area2
17    14                                               0  area2
18    18                                               0  area2

In [5]:
edges = pd.read_csv("check_import_edges.csv", na_filter=False)
edges

mark  father_mark
0      2            1
1      3            1
2      4            2
3      5            3
4      6            4
5      6            7
6      7            6
7      8            6
8      6            8
9      9            8
10     9           15
11    16            9
12    16           17
13    17           16
14    10           17
15    11           10
16    11            5
17    19           11
18    19           12
19    19           14
20    12           19
21    12           13
22    13           14
23    13           12
24    14           19
25    14           13
26    18           14

### Perturb a node

In order to perturb an element, we are going to use the function `simulate_element_perturbation`, which takes as an argument the list of the nodes that we want to damage.

In [6]:
F.simulate_element_perturbation(["1"])

DEBUG:root:In the graph are present 19 nodes
DEBUG:root:The graph is dense, density = 0.07894736842105263
DEBUG:root:BEST: {'2': True, '3': True}, with fitness: -20.0
DEBUG:root:Visited: {'1'}
DEBUG:root:Node: 1
DEBUG:root:Predecessors: []
DEBUG:root:Broken: ['1']
DEBUG:root:Visited: {'3', '1'}
DEBUG:root:Node: 3
DEBUG:root:Node 3 visited, fault resistant node
DEBUG:root:Visited: {'3', '1', '2'}
DEBUG:root:Node: 2
DEBUG:root:Node 2 visited, fault resistant node
DEBUG:root:In the graph are present 18 nodes
DEBUG:root:The graph is dense, density = 0.08169934640522876


The execution of the element perturbation gives us information about what is going on.
The size of the integer graph is reported (19 nodes), and depending on the graph density different methods for shortest path calculation are adopted. In particular, for dense graphs like the one in this tutorial, Floyd-Warshall algorithm is adopted.

The debug output gives us an idea of what happens when a node is perturbed. Node 1 has no fault resistance of any kind; hence, it gets broken. After that, the perturbation gets propagated on all neighboring nodes, namely nodes 2 and 3 in our case. They are visited, but not affected by the damage, being fault resistant.

Since node 2 and 3 are fault resistant, the best state identified for the switches by the genetic algorithm is still the one with switches closed, since it implies the smallest number of manual actions with respect to the initial condition. We are going to give a more detailed explanation on switches activation on a later tutorial. 

### Output

The simulation of the damage generates a few output files, which give us a diagnostics of the fault that just happened.

Output file `element_perturbation.csv` lists the new status of the elements the new status of the elements (active, not-active) and the areas in which the elements are located (affected, not affected) as well as the new status of elements that have been operated to stop the propagation of the perturbation or to open new paths in case of unavailabilty of the default ones. 
The efficiency (*nodal efficiency*, *local efficiency*, *global efficiency*) and closeness indices (*closeness_centrality*, *betweenness_centrality*, *indegree_centrality*) are then recalculated.

In [7]:
elements = pd.read_csv("element_perturbation.csv", na_filter=False)
elements

mark description init_status final_status mark_status  \
0      1                                       NOT_ACTIVE   
1      2                       1          1.0      ACTIVE   
2      3                       1          1.0      ACTIVE   
3      4                                           ACTIVE   
4      5                                           ACTIVE   
5      6                                           ACTIVE   
6      7                                           ACTIVE   
7      8                                           ACTIVE   
8      9                                           ACTIVE   
9     15                                           ACTIVE   
10    16                                           ACTIVE   
11    17                                           ACTIVE   
12    10                                           ACTIVE   
13    11                                           ACTIVE   
14    19                                           ACTIVE   
15    12                                           ACTIVE   
16    13                                           ACTIVE   
17    14                                           ACTIVE   
18    18                                           ACTIVE   

    perturbation_resistant   area status_area  closeness_centrality  \
0                        0  area1     DAMAGED              0.000000   
1                        1  area1     DAMAGED              0.055556   
2                        1  area1     DAMAGED              0.055556   
3                        1  area1     DAMAGED              0.074074   
4                        1  area1     DAMAGED              0.074074   
5                        0  area4   AVAILABLE              0.173611   
6                        0  area4   AVAILABLE              0.115741   
7                        0  area4   AVAILABLE              0.115741   
8                        0  area3   AVAILABLE              0.143275   
9                        0  area3   AVAILABLE              0.000000   
10                       0  area3   AVAILABLE              0.160714   
11                       0  area3   AVAILABLE              0.125000   
12                       0  area3   AVAILABLE              0.120773   
13                       0  area2   AVAILABLE              0.173868   
14                       0  area2   AVAILABLE              0.222994   
15                       0  area2   AVAILABLE              0.186693   
16                       0  area2   AVAILABLE              0.160556   
17                       0  area2   AVAILABLE              0.186693   
18                       0  area2   AVAILABLE              0.173077   

    betweenness_centrality  indegree_centrality  outdegree_centrality  \
0                 0.000000             0.000000              0.111111   
1                 0.051613             0.055556              0.055556   
2                 0.045161             0.055556              0.055556   
3                 0.129032             0.055556              0.055556   
4                 0.077419             0.055556              0.055556   
5                 0.270968             0.166667              0.111111   
6                 0.000000             0.055556              0.055556   
7                 0.283871             0.055556              0.111111   
8                 0.367742             0.111111              0.055556   
9                 0.000000             0.000000              0.055556   
10                0.374194             0.111111              0.055556   
11                0.367742             0.055556              0.111111   
12                0.348387             0.055556              0.055556   
13                0.419355             0.111111              0.055556   
14                0.380645             0.166667              0.111111   
15                0.103226             0.111111              0.111111   
16                0.000000             0.111111              0.111111   
17                0.109677             0.111111 

There is a lot of information contained in this output file. We would like to focus on some.
The status of the nodes contained in `mark_status` and `status_area` reflect the consequences of the damage: broken nodes are defined as `NOT_ACTIVE`, while areas that contain broken nodes are labeled as `DAMAGED`.

In [8]:
elements[['mark_status', 'status_area']]

mark_status status_area
0   NOT_ACTIVE     DAMAGED
1       ACTIVE     DAMAGED
2       ACTIVE     DAMAGED
3       ACTIVE     DAMAGED
4       ACTIVE     DAMAGED
5       ACTIVE   AVAILABLE
6       ACTIVE   AVAILABLE
7       ACTIVE   AVAILABLE
8       ACTIVE   AVAILABLE
9       ACTIVE   AVAILABLE
10      ACTIVE   AVAILABLE
11      ACTIVE   AVAILABLE
12      ACTIVE   AVAILABLE
13      ACTIVE   AVAILABLE
14      ACTIVE   AVAILABLE
15      ACTIVE   AVAILABLE
16      ACTIVE   AVAILABLE
17      ACTIVE   AVAILABLE
18      ACTIVE   AVAILABLE

Moreover, the nodes in the GeneralGraph are labeled as *SOURCE*, *HUB* or *USER*, and commodities flow from *SOURCE* to *USER*, passing through *HUB* nodes. In order to better understand this concept, let us load another output file generated by the perturbation simulation, `service_paths_element_perturbation.csv`, which reports the paths (all paths and shortest paths), if any, that connect *SOURCE* and *USER* elements (paths are calculated before and after perturbing one or multiple elements).

In [9]:
paths = pd.read_csv("service_paths_element_perturbation.csv", na_filter=False)
paths

from  to  original_shortest_path_length  \
0     1  18                            6.0   
1    15  18                            8.0   

                              original_shortest_path  \
0            ['1', '3', '5', '11', '19', '14', '18']   
1  ['15', '9', '16', '17', '10', '11', '19', '14'...   

                                original_simple_path  \
0  [['1', '2', '4', '6', '8', '9', '16', '17', '1...   
1  [['15', '9', '16', '17', '10', '11', '19', '12...   

   original_pair_efficiency   ids   area final_shortest_path_length  \
0                  0.166667   118                                     
1                  0.125000  1518  area3                        8.0   

                                 final_shortest_path  \
0                                                      
1  ['15', '9', '16', '17', '10', '11', '19', '14'...   

                                   final_simple_path final_pair_efficiency  
0                                                                           
1  [['15', '9', '16', '17', '10', '11', '19', '12...                 0.125

Since node 1 is broken, there is no final path between node 1 and node 18, while it still exists between nodes 15 and 18. Due to the damage of node 1, we can also see that the total final service at the *USER* 18 is smaller than the original one, lowered by 1 which is the service granted by the *SOURCE* represented by node 1, which is no more available.

In [10]:
elements[['original_service', 'final_service']].loc[18]

original_service      3
final_service       2.0
Name: 18, dtype: object